# Compute gene-pair proximities using random walk with restart

Export a table where the most proximal genes for each gene are exported.

Opened [this issue](https://github.com/sknetwork-team/scikit-network/issues/347) to see whether a more efficient implementation exists.

The resulting dataset `combined-score-top-proximities.tsv.xz` can be large. Users might want to filter the dataset using a command-line script for convenience, like:

```shell
# filter combined-score-top-proximities.tsv.xz with max_proximal_genes set to 50
curl --location --silent \
  https://github.com/related-sciences/string-protein-network/raw/2ac8e35e7983fa2d762142e7f287ade3bc8cfb63/data/proximities/combined-score-top-proximities.tsv.xz \
  | awk 'NR==1 || ($6 <= 50)' \
  | gzip --no-name >| combined-score-top-proximities-filtered.tsv.gz
```

In [1]:
import numpy as np
import pandas as pd
import scipy.sparse
import sknetwork as skn
import tqdm.notebook

from utils import get_protein_info_df

In [2]:
# parameters
max_proximal_genes = 100
proximity_threshold = 1.0
path = "data/proximities/combined-score-top-proximities.tsv.xz"

In [3]:
gene_df = get_protein_info_df()
gene_df.head(2)

,index,protein_external_id,preferred_name,protein_size,annotation
0,0,9606.ENSP00000000233,ARF5,180,ADP-ribosylation factor 5; GTP-binding protein...
1,1,9606.ENSP00000000412,M6PR,277,Cation-dependent mannose-6-phosphate receptor;...


In [4]:
matrix = scipy.sparse.load_npz("data/score-matrices/combined_score.sparse.npz")
matrix = matrix / 1000  # rescale STRING scores to be from 0 to 1

In [5]:
def get_rwr_scores(seeds):
    pagerank = skn.ranking.PageRank(damping_factor=0.85)
    np.random.seed(0)  # set seed in case there is some non-determinism
    scores = pagerank.fit_transform(adjacency=matrix, seeds=seeds)
    scores /= scores.mean()  # rescale scores to have a mean of 1
    return scores

In [6]:
nrows = matrix.shape[0]
nrows

19566

In [7]:
null_scores = get_rwr_scores(seeds=np.ones(nrows))
null_scores

array([2.15830465, 1.25733971, 1.98393592, ..., 1.        , 1.        ,
       0.41587283])

In [8]:
%%time
iterrows = tqdm.notebook.tqdm(gene_df.iterrows(), total=len(gene_df))
kwargs_chunk = dict(mode="a", header=False)
for i, row in iterrows:
    scores = get_rwr_scores(seeds={i: 1.0})
    df = pd.DataFrame(dict(
        gene_a=row.preferred_name,
        gene_b=gene_df.preferred_name,
        score=(1000 * np.asarray(matrix[i].todense()).flatten()).astype(int),
        proximity=scores,
        proximity_corrected=scores - null_scores,
    ))
    df = (
        df
        .query("proximity_corrected >= @proximity_threshold and gene_a != gene_b")
        .sort_values(["proximity_corrected"], ascending=False)
    )
    df = df.iloc[:max_proximal_genes, :]
    df["rank"] = df.proximity_corrected.rank(method="first", ascending=False).astype(int)
    kwargs = kwargs_chunk if i else {}
    df.to_csv(path, sep="\t", index=False, **kwargs, float_format="%.3g")


CPU times: user 4h 38min 3s, sys: 32min 37s, total: 5h 10min 40s
Wall time: 5h 10min 33s


In [9]:
df.head(5)

,gene_a,gene_b,score,proximity,proximity_corrected,rank
1517,OR6Q1,GNGT1,900,141.113393,134.955583,1
19077,OR6Q1,GNB1,900,142.026442,134.765930,2
7976,OR6Q1,GNAL,900,139.234503,134.187622,3
9549,OR6Q1,RTP2,902,136.466782,132.641310,4
6176,OR6Q1,RTP1,900,136.308976,132.444731,5
